# Scotch data
* Present interesting observation from the dataset
* Pick one Whiskey of choice, find similar options and explain model choice

*The data consists of multiple charatacteristics of chosen whiskies, including Jackson's five features, distilleries score, region, geo coordinates. The author considers the approach to classify the whisky types by taste profile (Jackson features), then apply the clusters on a map with their associated region to verify if geographic location can impact whiskies' taste profile*

*Method: Unsupervised learning: no labels provided, we want to group whiskies type by their specific feature, then validate with other attributes.*

*Algorithm: K- mean, iterates through a range of cluster (maximum equal number of regions) to get the best performing K.*
*With the result we then cross compare with data to see what other attribute closely correlate with Taste profile.*

## import useful modules

In [6]:
import pandas as pd
import numpy as no
import matplotlib.pyplot as plt

## Import and glimpse at data

In [93]:
df= pd.read_excel("scotch.xlsx")
df.head()
df.shape

/Users/tochi/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/tochi/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


(112, 85)

## Clean data

* Only 109 whisky types, remove the last invalid 2 rows

In [94]:
df=df[1:-2] #Remove the last 2 rows
df.shape

(109, 85)

* Get whisky types

In [138]:
whisky_type= df['Unnamed: 0'][1::]
whisky_type # view types

2          Aberlour
3           Ardberg
4           Ardmore
5      Auchentoshan
6          Aultmore
           ...     
105       Tobermory
106         Tomatin
107       Tomintoul
108         Tormore
109    Tullibardine
Name: Unnamed: 0, Length: 108, dtype: object

In [139]:
taste_profile=df.iloc[:,1:70] #extract only columns contain taste profile (68 columns according to data description)
taste_profile

,Unnamed: 1,color,color.1,color.2,color.3,color.4,color.5,color.6,color.7,color.8,...,FIN.9,FIN.10,FIN.11,FIN.12,FIN.13,FIN.14,FIN.15,FIN.16,FIN.17,FIN.18
1,Aberfeldy,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Aberlour,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Ardberg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Ardmore,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Auchentosh,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Tobermory,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
106,Tomatin,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
107,Tomintoul,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
108,Tormore,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


,Unnamed: 1,color,color.1,color.2,color.3,color.4,color.5,color.6,color.7,color.8,...,FIN.9,FIN.10,FIN.11,FIN.12,FIN.13,FIN.14,FIN.15,FIN.16,FIN.17,FIN.18
1,Aberfeldy,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Aberlour,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Ardberg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Ardmore,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Auchentosh,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Tobermory,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
106,Tomatin,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
107,Tomintoul,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
108,Tormore,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [132]:
taste_profile.info

<bound method DataFrame.info of      Unnamed: 1 color color.1 color.2 color.3 color.4 color.5 color.6 color.7  \
1     Aberfeldy     0       1       0       0       0       0       0       0   
2      Aberlour     0       0       0       0       0       0       0       0   
3       Ardberg     0       0       0       0       0       0       0       0   
4       Ardmore     0       0       0       0       1       0       0       0   
5    Auchentosh     0       0       0       0       1       0       0       0   
..          ...   ...     ...     ...     ...     ...     ...     ...     ...   
105   Tobermory     0       0       0       0       0       0       0       1   
106     Tomatin     0       0       0       0       0       0       0       1   
107   Tomintoul     0       0       0       0       0       0       0       1   
108     Tormore     0       0       0       0       0       0       0       1   
109  Tullibardi     0       0       0       0       0       1       0       0